In [69]:
import os
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import pandas as pd
import time
path = "alt.atheism/"

## Lab 2

Download and pre-process a large, publicly available text corpus. The preprocessing includes tokenization, normalization (e.g. convert to lowercase, remove non-alphanumeric chars, numbers, etc.). Output the preprocessing results as a text file, each line containing a single document (e.g. news article, sentence, yelp review, etc), with normalized tokens separated by a single white space. For example, the output file could look something like this:

_this is my first normalized sentence or document_

_this is my second sentence note that there are no numbers or punctuation_

_this is my third example note that all tokens are separated by a single white space_


In [11]:
text = []
for file in os.listdir(path):
    f = open(path + file, 'r', encoding = 'utf8', errors='ignore')
    text.append(f.read())
    f.close()

In [15]:
tokenized = []
for t in text:
    tokenized.append(nltk.word_tokenize(t))

In [24]:
normalized = []
for tokenized_text in tokenized:
    alphanum = [i.lower() for i in tokenized_text if i.isalpha()]
    normalized.append(alphanum)

In [26]:
with open('output.txt', 'w') as f:
    for doc in normalized:
        for token in doc:
            f.write(token+' ')
        f.write('\n')

Generate word2vec embeddings of your corpus using the original word2vec C implementation (https://code.google.com/archive/p/word2vec/ (Links to an external site.), https://github.com/tmikolov/word2vec (Links to an external site.)) or the python gensim implementation of the same algorithm (https://radimrehurek.com/gensim/models/word2vec.html (Links to an external site.)). Note that the gensim library can load/use word2vec embeddings generated with the original word2vec C implementation, which appears to be faster.

In [37]:
cbow_win5_emb100 = Word2Vec(normalized, size=100, window=5, min_count=3, sg=0, workers=4)

In [41]:
vector = cbow_win5_emb100.wv['computer']
vector

array([-2.64371455e-01,  1.09389460e+00, -2.35319510e-02, -1.48205966e-01,
        2.47048676e-01,  2.20061377e-01, -9.14129913e-01,  3.32614481e-01,
       -1.69367999e-01,  5.04543073e-04,  4.66485709e-01, -4.64058220e-02,
        4.26111341e-01, -9.72216010e-01, -5.31177640e-01, -1.14488229e-01,
       -4.87735420e-01, -7.20510781e-02, -1.47180334e-01,  1.46158412e-01,
        2.16246963e-01, -1.30026370e-01,  4.16960746e-01, -7.19683647e-01,
        9.12260637e-02, -1.94032807e-02, -3.59897882e-01, -4.25839238e-02,
        1.32670447e-01,  1.75703883e-01,  5.56196988e-01,  1.51722178e-01,
        2.31242329e-01,  4.45046186e-01,  5.82655594e-02, -1.82127580e-01,
        1.82748541e-01,  5.24740033e-02, -5.84662072e-02,  4.11371648e-01,
        5.37734747e-01, -4.20818001e-01, -4.15041178e-01,  9.58143473e-02,
        9.44186926e-01, -3.53702933e-01, -6.45911694e-01,  2.79962659e-01,
       -7.27420866e-01,  4.47494209e-01,  7.93027639e-01, -5.35342135e-02,
        6.73051834e-01, -

## Homework 2
After completing the lab assignment, experiment with 2 or more sets of word2vec model parameters, for example, different embedding sizes, CBOW vs. skip-gram models, etc. Provide a paragraph or two of qualitative evaluation of your embeddings (no need to provide a quantitative evaluation). For example, you can evaluate the embeddings by hand-picking ~10 words and manually reviewing/evaluating their closest neighbors in terms of cosine similarity.

In [70]:
start = time.time()
cbow_win5_emb100 = Word2Vec(normalized, size=100, window=5, min_count=3, sg=0, workers=4)
print("word2vec using CBOW, window size 5, 100 embedding size takes %s"%(time.time()-start))

start = time.time()
cbow_win10_emb150 = Word2Vec(normalized, size=150, window=10, min_count=3, sg=0, workers=4)
print("word2vec using CBOW, window size 10, 150 embedding size takes %s"%(time.time()-start))

start = time.time()
skip_win5_emb100 = Word2Vec(normalized, size=100, window=5, min_count=3, sg=1, workers=4)
print("word2vec using skip-gram, window size 5, 100 embedding size takes %s"%(time.time()-start))

word2vec using CBOW, window size 5, 100 embedding size takes 1.85636305809021
word2vec using CBOW, window size 10, 150 embedding size takes 1.9655609130859375
word2vec using skip-gram, window size 5, 100 embedding size takes 3.0176360607147217


In [59]:
def get_similar_most_words(model, target_words, num_of_words=5):
    res = {}
    for i in target_words:
        res[i] = model.wv.most_similar(i)[:num_of_words]
    return res

In [62]:
target_words = ['god', 'human', 'science', 'data', 'earth', 'play', 'physics', 'computer', 'study', 'myth']

In [65]:
pd.DataFrame(get_similar_most_words(cbow_win5_emb100, target_words))

,god,human,science,data,earth,play,physics,computer,study,myth
0,"(fact, 0.9543033838272095)","(statements, 0.9881774187088013)","(basis, 0.9386113882064819)","(music, 0.9981762170791626)","(fatwa, 0.9880374670028687)","(explaining, 0.9964451193809509)","(denying, 0.9943763017654419)","(dept, 0.9949401617050171)","(tired, 0.9958769679069519)","(treaty, 0.9931637048721313)"
1,"(one, 0.9418246150016785)","(evil, 0.9864280819892883)","(mathematically, 0.9318086504936218)","(atheisten, 0.9981348514556885)","(disciples, 0.9867024421691895)","(objects, 0.9958028793334961)","(expectation, 0.9934391379356384)","(university, 0.9938284158706665)","(explaining, 0.9957641363143921)","(violation, 0.9913896322250366)"
2,"(existence, 0.9378500580787659)","(animals, 0.9853005409240723)","(nervous, 0.931076169013977)","(american, 0.9980249404907227)","(ghost, 0.9860132932662964)","(tired, 0.9957259893417358)","(tree, 0.9927438497543335)","(wisconsin, 0.9929911494255066)","(fully, 0.9955913424491882)","(symbol, 0.9910142421722412)"
3,"(statement, 0.9377932548522949)","(religions, 0.9850634336471558)","(atheism, 0.9214677214622498)","(association, 0.9977816343307495)","(price, 0.9855170249938965)","(study, 0.9955664277076721)","(rule, 0.9919647574424744)","(austin, 0.9912014007568359)","(play, 0.9955663084983826)","(obtains, 0.9909652471542358)"
4,"(sense, 0.9355951547622681)","(very, 0.9825187921524048)","(based, 0.9207810163497925)","(radio, 0.9969357848167419)","(holy, 0.9851782321929932)","(experiences, 0.9950669407844543)","(string, 0.9909290671348572)","(cs, 0.9911085367202759)","(instructions, 0.9954851269721985)","(mathematics, 0.9909148216247559)"


In [66]:
pd.DataFrame(get_similar_most_words(cbow_win10_emb150, target_words))

,god,human,science,data,earth,play,physics,computer,study,myth
0,"(one, 0.958855390548706)","(explanations, 0.991916835308075)","(origins, 0.9702590703964233)","(required, 0.9971127510070801)","(humanity, 0.993659496307373)","(besides, 0.9975113868713379)","(rule, 0.9848600029945374)","(ang, 0.9939432144165039)","(rights, 0.9962847232818604)","(infiltrated, 0.9972487688064575)"
1,"(fact, 0.9571866989135742)","(real, 0.9895440340042114)","(basis, 0.9225066900253296)","(animal, 0.9964852333068848)","(age, 0.9922342300415039)","(somebody, 0.9973956346511841)","(k, 0.9843041300773621)","(computing, 0.9928671717643738)","(pursuit, 0.996250569820404)","(graham, 0.9947094917297363)"
2,"(existence, 0.9555343389511108)","(flawed, 0.9885271191596985)","(fiable, 0.9224928617477417)","(traditional, 0.9964069724082947)","(design, 0.9908194541931152)","(place, 0.9968032836914062)","(unbelief, 0.9817589521408081)","(az, 0.9904786944389343)","(running, 0.9960426092147827)","(stoesser, 0.9946727752685547)"
3,"(belief, 0.9450945258140564)","(course, 0.9877570867538452)","(swansont, 0.9208178520202637)","(independently, 0.9960430264472961)","(slaves, 0.9904066920280457)","(forms, 0.9961977005004883)","(preying, 0.9807174801826477)","(blacksburg, 0.9901084899902344)","(forced, 0.9954566955566406)","(joseph, 0.9942197799682617)"
4,"(no, 0.938841700553894)","(religions, 0.9875946044921875)","(case, 0.9105755090713501)","(golden, 0.9959813356399536)","(formal, 0.9892932176589966)","(having, 0.9957211017608643)","(dream, 0.9798636436462402)","(baarnie, 0.9892265796661377)","(stop, 0.9949263334274292)","(brandt, 0.9938904047012329)"


In [67]:
pd.DataFrame(get_similar_most_words(skip_win5_emb100, target_words))

,god,human,science,data,earth,play,physics,computer,study,myth
0,"(existence, 0.716551661491394)","(specific, 0.9142832159996033)","(percent, 0.6485558748245239)","(board, 0.9568888545036316)","(orbit, 0.8551596999168396)","(translate, 0.9595744609832764)","(rule, 0.7968345880508423)","(carnegie, 0.9554585218429565)","(consensus, 0.9646948575973511)","(lies, 0.9380513429641724)"
1,"(believing, 0.7146002650260925)","(result, 0.9074665904045105)","(origins, 0.64286208152771)","(medical, 0.9446496963500977)","(starts, 0.8529125452041626)","(spirit, 0.9566286206245422)","(pihatie, 0.7754154205322266)","(dept, 0.9526942372322083)","(fantasy, 0.9642175436019897)","(thank, 0.9321744441986084)"
2,"(eternal, 0.7083272933959961)","(changing, 0.9003133773803711)","(mixture, 0.6422374844551086)","(separation, 0.9345406293869019)","(heavens, 0.8321981430053711)","(interpret, 0.9552294015884399)","(boston, 0.7470213174819946)","(tech, 0.9524559378623962)","(powers, 0.9625577330589294)","(average, 0.929646372795105)"
3,"(satan, 0.7078456878662109)","(physical, 0.8995215892791748)","(computer, 0.6367412209510803)","(u, 0.9291795492172241)","(sun, 0.8060435056686401)","(trap, 0.9551805257797241)","(charlie, 0.7366487383842468)","(mellon, 0.9520890712738037)","(naturally, 0.9621647596359253)","(complaining, 0.9265915751457214)"
4,"(exists, 0.6709640026092529)","(raw, 0.898966908454895)","(guaranteed, 0.632087230682373)","(andrews, 0.928818941116333)","(night, 0.7860565781593323)","(literally, 0.9520004987716675)","(game, 0.7343603372573853)","(virginia, 0.9350047707557678)","(fully, 0.9605510234832764)","(incident, 0.9225119352340698)"


Read the required embeddings papers (word2vec, Bert, optionally Elmo) and compare the approaches in less than one page, preferably in a table format. You are free to decide what aspects to compare.  For example, you can include information such as: learning model details, summary of word context approaches, corpus size requirements, computational requirements, ease of installation/use of source code, date of publication and number of google scholar citations :), … etc.